In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="quick-starts-ws-133933")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133933
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-133933


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
### YOUR CODE HERE ###

cpu_cluster_name = "computer-project-one"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except:
    print("Creating new Compute Target...")
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it

Running


In [3]:
from azureml.widgets import RunDetails
import shutil
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig, Experiment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os


# Specify parameter sampler
ps = RandomParameterSampling({
    #"--penalty": choice('11', 'l2', 'elasticnet', 'none'),
    "--C" : uniform(0.01, 2),
    "--max_iter" : choice(50, 100, 150, 200, 250, 300)
    #"--solver" : choice('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = '.'
# os.makedirs(script_folder, exist_ok=True)
# shutil.copy('./train.py', script_folder)

# Create a SKLearn estimator for use with train.py

est = SKLearn(
    source_directory= script_folder,
    compute_target= cpu_cluster,
    entry_script= "train.py"
)


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [ ]:
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(
    estimator=est, 
    hyperparameter_sampling=ps, 
    policy=policy,
    primary_metric_name='accuracy', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=20,
    max_concurrent_runs=4
    )

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_de67c126-1f4e-4e7d-ba3f-3eeb1c5ebcb2
Web View: https://ml.azure.com/experiments/quick-starts-ws-133933/runs/HD_de67c126-1f4e-4e7d-ba3f-3eeb1c5ebcb2?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-133933/workspaces/quick-starts-ws-133933

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-08T04:26:26.649801][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-08T04:26:26.828499][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2021-01-08T04:26:26.150876][API][INFO]Experiment created<END>\n"<START>[2021-01-08T04:26:28.2224895Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [45]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Run ID: ', best_run.id)
print('Best Run Accuracy: ', best_run_metrics['accuracy'])
print('Best Run Learning Rate: ', parameter_values[3])


Best Run ID:  HD_e6a3ceb9-68f5-4070-91bf-da354491d133_5
Best Run Accuracy:  0.9138088012139606
Best Run Learning Rate:  50


In [49]:
# registering the best model
best_run.get_file_names()

best_run.register_model(model_name="Bank_Marketing_Best_model", model_path ="./outputs/parta_model.joblib")

best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-133828,HD_e6a3ceb9-68f5-4070-91bf-da354491d133_5,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

url_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

In [ ]:
# checking the uploaded data
ds_dataframe = ds.to_pandas_dataframe()
ds_dataframe.head()

In [98]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)



In [81]:

# from sklearn.model_selection import train_test_split
# import pandas as pd
# from azureml.core import Dataset

# x_train, x_test = train_test_split(x, test_size = 0.2)
# pd.DataFrame(x).to_csv("train.csv")


#OPTIONAL : SAVING TRAINING DATA TO THE DATASTORE
# deafult_store = ws.get_default_datastore()
# deafult_store.upload(src_dir="./", target_path="./")
# train_data = Dataset.Tabular.from_delimited_files(path=[deafult_store.path("./train.csv")])

In [99]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df,
    label_column_name="y",
    n_cross_validations=5,
    compute_target = cpu_cluster)

print("AutoML config created,.")


AutoML config created,.


In [100]:
experiment = Experiment(ws, 'Bank_Marketing_AutoML')
print("Experiment created")

Experiment created


In [102]:
# Submit your automl run

### YOUR CODE HERE ###
auto_ml_run = experiment.submit(automl_config, show_output = True)
RunDetails(auto_ml_run).show()
auto_ml_run.wait_for_completion(show_output=True)

ConfigException: ConfigException:
	Message: Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###